In [5]:
# Basic imports
import scipy, pandas as pd, numpy as np
from matplotlib import pyplot as plt

In [4]:
# set the constants
CUTOFF = 37.78
DAYS = 3

LEAST_FILTER = 26.67

THRESHOLDS = [26.67, 32.22, 37.78]

In [6]:
# reshape, remove unnecessary columns and add statistical filter + threshold
def proc(frame, upper_threshold, year):
    processed = frame.copy()
    for name in list(frame):
        if name[:4] != "tmax" and name != "GEOID20":
            processed = processed.drop(name, axis=1)
    
        
    stats = processed.set_index('GEOID20')
    stats = stats.apply(pd.DataFrame.describe, axis=1)
    stats["upper"] = stats["mean"]+1.645*stats["std"]

    processed = processed.melt(id_vars='GEOID20', var_name='Date', value_name='Temp')
    
    # conversion to Python datetime objects
    processed['Date'] = processed['Date'].map(lambda x: x.lstrip('tmax')) + str(year)
    processed['Date'] = pd.to_datetime(processed['Date'], format='%b%d%Y')

    # Set + sort by multi-level index using 'GEOID20' and 'Date'
    processed.set_index(['GEOID20', 'Date'], inplace=True)
    processed.sort_index(inplace=True)

    thresholds = stats.copy()
    thresholds.drop(columns=['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max'], inplace=True)
    
    # for upper here we currently have this as 32.2
    thresholds['static']= upper_threshold
    
    merged = processed.reset_index().merge(thresholds.reset_index(), on='GEOID20', how='left').set_index(['GEOID20', 'Date'])

    return merged

In [7]:
# function to create a (not indexed) boolean series which tells you whether an entry is part of a consecutive series of length n
def consec_count_n(series, n):
    temp = []
    c = 0
    for i in range(len(series.values)):
        if series.values[i] == False:
            c = 0
        else:
            c += 1
        temp.append(c)
    
    result = []
    pass_thres = False
    for i in range(len(temp)):
        if temp[-(i+1)] >= n:
            pass_thres = True
            result.append(True)
        elif temp[-(i+1)] != 0 and pass_thres:
            result.append(True)
        else:
            pass_thres = False
            result.append(False)
    result = result[::-1]

    return pd.Series(result)

In [8]:
# function to check if the contig_mask (consec_count function) is working
def check(frame, contig_mask):
    threshold_mask = (frame['Temp'] > frame['upper']) | (frame['Temp'] > frame['static'])

    result = pd.DataFrame(contig_mask)

    result.reset_index()
    result.columns = ["contiguous"]
    result.index = frame.index

    check = pd.DataFrame(result)
    check.insert(1, "true-false", threshold_mask)
    
    with pd.option_context('display.max_rows', None,):
        print(check)

In [9]:
# function which returns a filtered version of a given dataframe which only includes the entries which are part of a consecutive series of length n Trues for a boolean mask
def consec_filter(frame, n):
    threshold_mask = (frame['Temp'] > frame['upper']) | (frame['Temp'] > frame['static'])
    
    contig_mask = pd.DataFrame(data=consec_count_n(threshold_mask, n))

    contig_mask.reset_index()
    contig_mask.columns = ["contiguous"]
    contig_mask.index = frame.index

    filtered_data = frame[frame.index.isin(contig_mask.index[contig_mask['contiguous']])]

    return filtered_data

In [10]:
# function to turn the dataframe of days into flat data
def flatten(frame):
    sequences = []

    # loop to find and append the sequences and their lengths to a new array
    for zip_code, group in frame.groupby(level='GEOID20'):
        start_date = None
        sequence_length = 0
        prev_date = None
        for date in group.index.get_level_values('Date'):
            if start_date is None:
                start_date = date
                sequence_length = 1
            elif prev_date is not None and (date - prev_date).days == 1:
                sequence_length += 1
            else:
                sequences.append({
                    'GEOID20': zip_code,
                    'Start_Date': start_date.date(),
                    'Sequence_Length': sequence_length
                })
                start_date = date
                sequence_length = 1
            prev_date = date
        if sequence_length > 0:
            sequences.append({
                'GEOID20': zip_code,
                'Start_Date': start_date.date(),
                'Sequence_Length': sequence_length
            })
    # flatten to 2d array
    result = [[seq['GEOID20'], seq['Start_Date'], seq['Sequence_Length']] for seq in sequences]

    return result

In [11]:
# function to do the whole process for a single year
def find_heatwaves(frame, cutoff, days, year):
    processed = proc(frame, cutoff, year)
    days = consec_filter(processed, days)
    result = flatten(days)
    return result

In [12]:
def mCounts(df, thresholds):
    # Extract year and month from datetime column
    df['Year'] = df.index.get_level_values('Date').year
    df['Month'] = df.index.get_level_values('Date').month

    # Initialize a dictionary to store counts for each threshold
    threshold_counts = {}

    # Iterate over each threshold
    for threshold in thresholds:
        # Filter the dataframe to get only entries exceeding the threshold
        exceed_threshold_df = df[df['Temp'] > threshold]
        
        # Group by zip code, year, and month and count the number of exceedances
        grouped_df = exceed_threshold_df.groupby(['GEOID20', 'Year', 'Month']).size().reset_index(name=f'days_below_{threshold}')
        
        # Convert the count values to integers
        grouped_df[f'days_below_{threshold}'] = grouped_df[f'days_below_{threshold}'].astype(int)

        # Store the counts for this threshold in the dictionary
        threshold_counts[threshold] = grouped_df

    # Ensure all combinations of 'GEOID20', 'Year', and 'Month' are present in each threshold's grouped DataFrame
    for threshold in thresholds:
        all_combinations = pd.MultiIndex.from_product([threshold_counts[threshold]['GEOID20'].unique(),
                                                    threshold_counts[threshold]['Year'].unique(),
                                                    threshold_counts[threshold]['Month'].unique()],
                                                    names=['GEOID20', 'Year', 'Month'])
        threshold_counts[threshold] = threshold_counts[threshold].set_index(['GEOID20', 'Year', 'Month']).reindex(all_combinations).fillna(0).reset_index()

    # Merge the dataframes for each threshold on zip code, year, and month
    result_df = threshold_counts[thresholds[0]]
    for threshold in thresholds[1:]:
        result_df = pd.merge(result_df, threshold_counts[threshold], on=['GEOID20', 'Year', 'Month'], how='outer').fillna(0)

    result_df = result_df.drop(columns=['Year'])
    
    result_df.sort_index()

    # Now result_df contains the desired format with zip code, year, month, and the count of exceedances
    return result_df

In [13]:
def mCountSort(mCount):
    sorted = mCount.sort_values(by=['GEOID20', 'Month'], kind='mergesort')

    sorted.reset_index
    sorted.set_index(['GEOID20', 'Month'])
    
    return sorted

In [51]:
# format the headers of the columns
columnNames = ['Zip_Code', 'Starting_Date', 'Days']

In [52]:
years = range(2006, 2023)

for year in years:
    frame = pd.read_csv("extracted_by_year/" + str(year) + "_extracted/tmax" + str(year) + ".csv")
    result = find_heatwaves(frame, CUTOFF, DAYS, year)
    np.savetxt("heatwave_" + str(year) + "_cutoff" + str(CUTOFF) + ".csv", result, fmt='%s', delimiter=', ', header=', '.join(columnNames))

: 

In [ ]:
# mCount section: find for each year the number of days per month various threholds are exceeded
years = range(2006, 2023)
columnNames = ['GEOID20', 'Month', 'days above 26.67', 'days above 32.22', 'days above 37.78']


for year in years:
    frame = pd.read_csv("extracted_by_year/" + str(year) + "_extracted/tmax" + str(year) + ".csv")
    # use least strict filter to create the consec filter dataframe that you will use for all other thresholds
    filter =  consec_filter(proc(frame, LEAST_FILTER, year), 1)
    result = mCountSort(mCounts(filter, THRESHOLDS))
    np.savetxt("monthcounts" + str(year) + ".csv", result, fmt='%s', delimiter=', ', header=', '.join(columnNames))
    